<a href="https://colab.research.google.com/github/TZuanazzi/Liver_Segmentation_NN/blob/main/gerador_de_imagem_e_ataque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# os.kill(os.getpid(), 9)


In [ ]:
# !pip install  torchmetrics
# !pip install  tqdm
# !pip install  torch
# !pip install  torchvision
# !pip install  TorchAudio
# !pip install  Cython
# !pip install  torchattacks
# !pip install  opencv-python

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import torch


class DresdenDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        basename = os.path.basename(image_dir)
        self.image_names = [filename for filename in os.listdir(image_dir) if filename.startswith("image")]
        self.label_names = [filename for filename in os.listdir(image_dir) if filename.startswith("mask")]

        # Sort the image and label names based on the numeric part extracted from filenames
        self.image_names.sort(key=lambda x: int(x[5:7]))  # Extract the two-digit number from "imageXX.png"
        self.label_names.sort(key=lambda x: int(x[4:6]))  # Extract the two-digit number from "maskXX.png"

        print("image_names:", self.image_names)
        print("label:", self.label_names)

    def __len__(self):
        return len(self.image_names)

    def classes(self):
        return torch.Tensor([0,1])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = np.array(Image.open(os.path.join(self.image_dir, self.image_names[idx])).convert('RGB'))
        label1 = np.array(Image.open(os.path.join(self.image_dir, self.label_names[idx])).convert('RGB'))
        # to use just three conditions, we create another label with np.zeros
        label = np.zeros(np.shape(label1), np.uint8)[:,:,0:2]
        label[:,:,0][label1[:,:,0]>125] = 1
        label[:,:,1][label1[:,:,0]<125] = 1

        dictionary = {'image0': image, 'image1': label}

        if self.transform is not None:
            dictionary = self.transform(dictionary)

        return dictionary


In [ ]:
import torch
import torch.nn as nn
# import time

# Since nn.Sequential does not handle multiple inputs, create mySequential to
# handle it, inhiriting from nn.Sequential
class mySequential(nn.Sequential):
    def forward(self, *inputs):
        for module in self._modules.values():
            if type(inputs) == tuple:
                inputs = module(*inputs)
            else:
                inputs = module(inputs)
        return inputs


class block_standard(nn.Module):
    #defining block expansion
    expansion: int = 1
    # To devide the 'out_channels' by 2 in the standard, we create this variab.
    out_multiply: int = 2

    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1, up=False):

        super(block_standard, self).__init__()

        self.up = up
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

        if self.up:
            self.conv1 = nn.ConvTranspose2d(in_channels, out_channels,
                                            kernel_size=stride, stride=stride)
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                                   stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x, long_skip=None):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if (self.identity_downsample is not None):
            identity = self.identity_downsample(identity)
        # if long_skip==None: print('long skip none')
        if long_skip is not None:
            x = torch.cat((x, long_skip), dim=1)
        x += identity
        x = self.relu(x)

        del identity

        return x, long_skip


class block_bottleneck(nn.Module):
    # defining block expansion
    expansion: int = 4
    # To devide the 'out_channels' by 2 in the standard, we create this variab.
    out_multiply: int = 1

    def __init__(self, in_channels, out_channels, identity_scale=None, stride=1, up=False):

        super(block_bottleneck, self).__init__()

        self.up = up
        self.in_channels = in_channels
        self.out_channels = out_channels

        if self.up:
            self.expansion = 2
        else:
            self.expansion = 4

        if self.up:
            self.conv1 = nn.ConvTranspose2d(in_channels, out_channels,
                                            kernel_size=stride, stride=stride)
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                   stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)

        if self.up:
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                                   stride=1, padding=1)
        else:
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                                   stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion,
                               kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_scale = identity_scale

    def forward(self, x, long_skip=None):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_scale is not None:
            identity = self.identity_scale(identity)
        # if long_skip==None: print('long skip none')
        if long_skip is not None:
            x = torch.cat((x, long_skip), dim=1)
        x += identity
        x = self.relu(x)

        del identity

        return x, long_skip


class UResNet(nn.Module): # [3, 4, 6, 3]

    def __init__(self, block, layers, image_channels, num_classes):

        super(UResNet, self).__init__()
        self.in_channels = 64
        # First Convolutions
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7,
                               stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.long_skip = []

        # ResNet Layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2)

        # ResNet Layers
        self.layer5 = self._make_layer(block, layers[3], out_channels=512, stride=2, up=True)
        self.layer6 = self._make_layer(block, layers[2], out_channels=256, stride=2, up=True)
        self.layer7 = self._make_layer(block, layers[1], out_channels=128, stride=2, up=True)
        self.layer8 = self._make_layer(block, layers[0], out_channels=64, stride=1)

        # Last Convolutions
        self.conv_last1 = nn.ConvTranspose2d(self.in_channels, 64,
                                             kernel_size=2, stride=2, padding=0)
        self.conv_last2 = nn.ConvTranspose2d(64*2, num_classes,
                                             kernel_size=2, stride=2, padding=0)
        self.bn2 = nn.BatchNorm2d(num_classes)
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        self.long_skip = [0, 0, 0, 0]
        self.long_skip[0] = x
        x = self.maxpool(x)

        x, temp = self.layer1(x, None)
        self.long_skip[1] = x
        x, temp = self.layer2(x, None)
        self.long_skip[2] = x
        x, temp = self.layer3(x, None)
        self.long_skip[3] = x
        x, temp = self.layer4(x, None)

        self.long_skip = self.long_skip[::-1]

        x, temp = self.layer5(x, self.long_skip[0])
        x, temp = self.layer6(x, self.long_skip[1])
        x, temp = self.layer7(x, self.long_skip[2])
        x, temp = self.layer8(x, None)
        x = self.conv_last1(x)
        x = self.bn1(x)
        x = torch.cat((x, self.long_skip[3]), dim=1)
        x = self.relu(x)
        x = self.conv_last2(x)
        x = self.bn2(x)
        x = self.Softmax(x)

        del self.long_skip, temp

        return x


    def _make_layer(self, block, num_residual_blocks,
                         out_channels, stride, up=False):
        identity_scale = None
        layers = []

        if up==False:
            if stride != 1 or self.in_channels != out_channels*block.expansion:
                identity_scale = mySequential(nn.Conv2d(self.in_channels,
                                                        out_channels*block.expansion,
                                                        kernel_size=1,
                                                        stride=stride),
                                              nn.BatchNorm2d(out_channels*block.expansion))

            layers.append(block(self.in_channels, out_channels,
                                identity_scale, stride))
            self.in_channels = out_channels*block.expansion

            for i in range(num_residual_blocks-1):
                layers.append(block(self.in_channels, out_channels))

        else:
            if stride != 1 or self.in_channels != out_channels*block.expansion:
                identity_scale = mySequential(nn.ConvTranspose2d(self.in_channels,
                                                                 out_channels*block.expansion,
                                                                 kernel_size=stride,
                                                                 stride=stride),
                                              nn.BatchNorm2d(out_channels*block.expansion))
            # This is to devide the 'out_channels' by 2 in the standard block
            # if you see, the output channels is really half of the value in
            # this block.
            out_channels = int(out_channels/block.out_multiply)
            layers.append(block(self.in_channels, out_channels,
                                identity_scale, stride, up=True))

            if stride==1 and up==True:
                self.in_channels = out_channels*2
            elif block.expansion == 1 and up==True:
                self.in_channels = out_channels*2
            else:
                self.in_channels = out_channels*4

            for i in range(num_residual_blocks-1):
                layers.append(block(self.in_channels, out_channels, up=True))

        return mySequential(*layers)

def UResNet18(in_channels=3, num_classes=3):
    return UResNet(block_standard, [2, 2, 2, 2], in_channels, num_classes)

def UResNet34(in_channels=3, num_classes=3):
    return UResNet(block_standard, [3, 4, 6, 3], in_channels, num_classes)

def UResNet50(in_channels=3, num_classes=3):
    return UResNet(block_bottleneck, [3, 4, 6, 3], in_channels, num_classes)

def UResNet101(in_channels=3, num_classes=3):
    return UResNet(block_bottleneck, [3, 4, 23, 3], in_channels, num_classes)

def UResNet152(in_channels=3, num_classes=3):
    return UResNet(block_bottleneck, [3, 8, 36, 3], in_channels, num_classes)


def test():
    net = UResNet34()
    # working sizes 224, 256, 288
    x = torch.randn(2, 3, 575, 575)
    if torch.cuda.is_available():
        y = net(x).to('cuda')
    else:
        y = net(x)
    print(y.shape)

if __name__ == '__main__':
    test()

# print('- Time taken:', time.time()-start)

torch.Size([2, 3, 576, 576])


In [ ]:
'''
This file is used together with the 'train.py' file to help in the training and
testing process with util functions.
'''
import torch
# from dataset import DresdenDataset
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.functional as tf
from torchvision.transforms import Compose
from torchvision.utils import save_image
from tqdm import tqdm
import random
from torchmetrics import Accuracy
from torchmetrics import F1Score
from torchmetrics import JaccardIndex
from torchmetrics import Recall
from torchmetrics import Precision
from torchmetrics import Specificity



# The next functions are functional transforms, used to apply functions in a way
# controled by the user. So we can apply, for example, in the data image and in
# the label image (so it is called deterministic, because we can determine the
# same transformation to be applied in more then one image). This is the unique
# way to apply the same transformation to more then one different image in torch

class ToTensor(object):
    '''Function to transform a ndarray in a tensor

    n: int (input)
        number of non-mask images to convert to tensor (the rest will be
        converted without scaling to [0.0,1.0])'''
    def __init__(self, n):
        self.n = n

    def __call__(self, images):
        for i, image in enumerate(images):
            if i < self.n:
                images[image] = tf.to_tensor(images[image])
            else:
                images[image] = torch.from_numpy(images[image])
                images[image] = torch.permute(images[image], (2,0,1))

        return images


class Rotate(object):
    '''Function to rotate an image, the input is a dictionary

    images: 'dictionary' (input)
        dictionary with images;
    limit: 'list'
        a list 'int' with smaller and larger angles to rotate (e.g. [0, 90]);
    p: 'float'
        probability to rotate;

    dictionary: 'dictionary' (output)
        dictionary with cropped images with keys 'image0', 'image1', etc.
    '''
    def __init__(self,**kwargs):
        # 'limit' is a 'list' that defines the lower and upper angular limits
        limit = kwargs.get('limit')
        if not limit: limit = [0, 360]
        self.limit = limit
        # 'p' is 'float' the probability to happen a rotate
        p = kwargs.get('p')
        if not p: p = 0.5
        self.p = p

    def __call__(self, images):
        if random.random() > 1-self.p:
            angle = random.randint(self.limit[0], self.limit[1])
            for i, image in enumerate(images):
                images[image] = tf.rotate(images[image], angle)

        return images


class CenterCrop(object):
    '''Function to center crop one or multiple images

    size: 'list' (input)
        input list with size (e.g. '[400,200]');
    images: 'dictionary' (input) (output)
        dictionary with images.
    '''
    def __init__(self, size):
        self.size = size


    def __call__(self, images):
        for image in images:
            images[image] = tf.center_crop(images[image], self.size)

        return images


class Resize(object):
    '''Function to resize one or multiple images

    size: 'list' (input)
        input list with size (e.g. '[400,200]');
    images: 'dictionary' (input) (output)
        dictionary with images.
    '''
    def __init__(self, size):
        self.size = size

    def __call__(self, images):
        for image in images:
            images[image] = tf.resize(images[image], self.size)

        return images


class FlipHorizontal(object):
    '''Horizontally flip images randomly

    p: 'float' (input)
        probability to flip (from 0.0 to 1.0).
    '''
    def __init__(self, p):
        self.p = p

    def __call__(self, images):
        if random.random() > 1-self.p:
            for image in images:
                images[image] = tf.hflip(images[image])

        return images


class FlipVertical(object):
    '''Vertically flip images randomly

    p: 'float' (input)
        probability to flip (from 0.0 to 1.0).
    '''
    def __init__(self, p):
        self.p = p

    def __call__(self, images):
        if random.random() > 1-self.p:
            for image in images:
                images[image] = tf.vflip(images[image])

        return images


class Normalize(object):
    '''Normalizing 'n' images of a given set of images

    n: int (input)
        number of images to normalize;
    mean: list (input)
        mean to normalize;
    std: list (input)
        stadard deviation to normalize.
    '''
    def __init__(self, n=1, mean=0.5, std=0.5):
        self.n = n
        self.mean = mean
        self.std = std

    def __call__(self, images):
        for i, image in enumerate(images):
            if i < self.n:
                images[image] = tf.normalize(images[image], self.mean, self.std)

        return images


class Affine(object):
    '''Affining images

    size: list (input)
        maximum higher and width to translate image (normally the image size);
    scale: float (input)
        scale to perform affine (between 0 and 1.0);
    p: float (input)
        probability to thange.'''
    def __init__(self, size=[0,0], scale=0.5, p=0.5):
        self.size = size
        self.scale = scale
        self.p = p

    def __call__(self, images):
        if random.random() > 1-self.p:
            angle = random.random()*self.scale*360
            shear = random.random()*self.scale*360
            translate = [i*random.random()*self.scale for i in self.size]
            for image in images:
                images[image] = tf.affine(images[image], angle=angle,
                                         translate=translate, scale=1-self.scale,
                                         shear=shear)
        return images


#%% Saving and Loading Checkpoints, getting the Data Loaders

# saving checkpoints
def save_checkpoint(state, filename='my_checkpoint.pth.tar'):
    print('\n- Saving Checkpoint...')
    torch.save(state, filename)

# loading checkpoints
def load_checkpoint(checkpoint, model, optimizer=None):
    print('\n- Loading Checkpoint...')
    model.load_state_dict(checkpoint['state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer'])

# getting loaders given directories and other informations
def get_loaders(train_image_dir,
                valid_percent,
                test_percent,
                batch_size,
                image_height,
                image_width,
                num_workers=1,
                pin_memory=True,
                val_image_dir=None,
                clip_valid=1.0,
                clip_train=1.0,
                clip_test=1.0):

    # first, defining transformations to be applied in the train images to be loaded
    transform_train_0 = Compose([ToTensor(n=1),
                                 Resize(size=[image_height, image_width]),
                                 # mean and std, obtained from Dresden Dataset
                                 # for segmentation
                                 Normalize(n=1, mean=[0.4338, 0.31936, 0.312387],
                                           std=[0.1904, 0.15638, 0.15657])]
                                )
    # defining the same, but for validation and testing images (can be different)
    transform_valid_0 = Compose([ToTensor(n=1),
                                 Resize(size=[image_height, image_width]),
                                 # defining again if validation dataset is dif.
                                 Normalize(n=1, mean=[0.4338, 0.31936, 0.312387],
                                           std=[0.1904, 0.15638, 0.15657])]
                                )

    # second, defining the number of transformations per directory in
    # 'train_image_dir' defines the data augmantation (1 for no augmentation
    # and 5 for 5 times augmentation)
    transformations_per_dataset = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    # transformations_per_dataset = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    #                                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
    # transformations_per_dataset = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
    #                                 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
    #                                 10, 10, 10, 10, 10, 10, 10, 10]

    # third, reading the dataset in a as a 'torch.utils.data.Dataset' instance.
    # it is only for images in 'train_image_dir[0]', further we will accounts
    # for the rest of the directories
    train_dataset = DresdenDataset(image_dir=train_image_dir[0],
                                  transform=transform_train_0)

    # concatenate the other directories in 'train_image_dir[:]' in a larger
    # 'torhc.utils.data.Dataset'. after we will concatenate more for augmentat.
    for n in range(1, len(train_image_dir)):
        dataset_train_temp = DresdenDataset(image_dir=train_image_dir[n],
                                           transform=transform_train_0)
        # to use 'train_dataset' here in right, we have to define it before
        train_dataset = torch.utils.data.ConcatDataset([train_dataset,
                                                        dataset_train_temp])

    # using part of the training data as test dataset
    test_dataset_size = int(test_percent*len(train_dataset))
    rest_size = int((1-test_percent)*len(train_dataset))
    if test_dataset_size+rest_size != len(train_dataset):
        rest_size += 1
    (test_dataset, _) = random_split(train_dataset, [test_dataset_size, rest_size],
                                     generator=(torch.Generator().manual_seed(40)))

    # defining the validation dataset, using part of the 'train_dataset', or
    # using a specific dataset for validation, if 'val_image_dir' is not 'None'
    if not val_image_dir:
        valid_dataset_size = int(valid_percent*len(train_dataset))
        train_dataset_size = int((1-valid_percent)*len(train_dataset))
        # adding one to train_dataset_size if 'int' operation removed it
        if valid_dataset_size+train_dataset_size != len(train_dataset):
            train_dataset_size += 1
        (train_dataset, valid_dataset) = random_split(train_dataset,
                                         [train_dataset_size, valid_dataset_size],
                                         generator=torch.Generator().manual_seed(20))
    else:
        valid_dataset = DresdenDataset(image_dir=val_image_dir[0],
                                      transform=transform_valid_0)
        for n in range(1, len(val_image_dir)):
            dataset_val_temp = DresdenDataset(image_dir=val_image_dir[n],
                                             transform=transform_valid_0)
            valid_dataset = torch.utils.data.ConcatDataset([valid_dataset,
                                                            dataset_val_temp])

    # concatenating the augmented data, in case 'transf..._per_dataset' > 1
    for n in range(0,len(train_image_dir)):
        for m in range(1, transformations_per_dataset[n]):
            # first we specify the transformation (depending on the 'm' value)
            if m < 2:
                transformation = Compose([ToTensor(n=1),
                                          Resize(size=[image_height, image_width]),
                                          # Mean and std, obtained from the dataset
                                          Normalize(n=1, mean=[0.4338, 0.31936, 0.312387],
                                                    std=[0.1904, 0.15638, 0.15657])]
                                          )
            else:
                transformation = Compose([ToTensor(n=1),
                                          Resize(size=[image_height, image_width]),
                                          # Mean and std, obtained from the dataset
                                          Normalize(n=1, mean=[0.4338, 0.31936, 0.312387],
                                                    std=[0.1904, 0.15638, 0.15657])]
                                          )
            # then we apply this transformation to read the dataset as 'torch.utils.data.Dataset'
            dataset_train_temp = DresdenDataset(image_dir=train_image_dir[n],
                                               transform=transformation)
            train_dataset = torch.utils.data.ConcatDataset([train_dataset, dataset_train_temp])

    # splitting the dataset, to deminish if 'clip_valid'<1 for fast testing
    if clip_train < 1:
        print('\n- Splitting Training Dataset ',clip_train*100,'%')
        train_mini = int(clip_train*len(train_dataset))
        temp_mini = int((1-clip_train)*len(train_dataset))
        if train_mini+temp_mini != len(train_dataset):
            temp_mini += 1
        (train_dataset, _) = random_split(train_dataset,[train_mini, temp_mini],
                                          generator=torch.Generator().manual_seed(40))
    if clip_valid < 1:
        print('\n- Splitting Validation Dataset ',clip_valid*100,'%')
        valid_mini = int(clip_valid*len(valid_dataset))
        temp_mini = int((1-clip_valid)*len(valid_dataset))
        if valid_mini+temp_mini != len(valid_dataset):
            temp_mini += 1
        (valid_dataset, _) = random_split(valid_dataset,[valid_mini, temp_mini],
                                          generator=torch.Generator().manual_seed(30))

    if clip_test < 1:
        print('\n- Splitting Testing Dataset ',clip_test*100,'%')
        test_mini = int(clip_test*len(test_dataset))
        temp_mini = int((1-clip_test)*len(test_dataset))
        if test_mini+temp_mini != len(test_dataset):
            temp_mini += 1
        (test_dataset, _) = random_split(test_dataset, [test_mini, temp_mini],
                                         generator=torch.Generator().manual_seed(50))

    # obtaining dataloader from the datasets defined above
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              num_workers=num_workers,
                              pin_memory=pin_memory, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                             num_workers=num_workers,
                             pin_memory=pin_memory)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size,
                              num_workers=num_workers,
                              pin_memory=pin_memory)

    return train_loader, test_loader, valid_loader


#%% Function to check accuracy

def check_accuracy(loader, model, loss_fn, output_labels,
                   device='cuda' if torch.cuda.is_available() else 'cpu',
                   **kwargs):
    # Here 'output_labels == 2' means binary. We use this conditional because
    # the model we are using returns a multilabel data, when we use this model
    # with 2 labels, it becomes binary.
    if output_labels == 2:
        accuracy = Accuracy(task='binary', num_labels=output_labels).to(device)
        f1score = F1Score(task='binary', num_labels=output_labels).to(device)
        intersection = JaccardIndex(task='binary', num_labels=output_labels).to(device)
        recall = Recall(task='binary', num_labels=output_labels).to(device)
        precision = Precision(task='binary', num_labels=output_labels).to(device)
        specificity = Specificity(task='binary', num_labels=output_labels).to(device)
    else:
        accuracy = Accuracy(task='multilabel', num_labels=output_labels).to(device)
        f1score = F1Score(task='multilabel', num_labels=output_labels).to(device)
        intersection = JaccardIndex(task='multilabel', num_labels=output_labels).to(device)
        recall = Recall(task='multilabel', num_labels=output_labels).to(device)
        precision = Precision(task='multilabel', num_labels=output_labels).to(device)
        specificity = Specificity(task='multilabel', num_labels=output_labels).to(device)
    # using model in evaluation
    model.eval()
    # if title is passed, use it before 'Check acc' and 'Got an accuracy...'
    title = kwargs.get('title')
    if title==None: title = ''
    else: title = title+': '
    # using tqdm.tqdm to show a progress bar
    loop = tqdm(loader, desc=title+'Check acc')

    with torch.no_grad():
        for dictionary in loop:
            image, label = dictionary
            x, y = dictionary[image], dictionary[label]
            x, y = x.to(device=device), y.to(device=device)
            y = y.float()
            pred = model(x)['out']
            # Here 'output_labels == 2' means binary. We use this way because
            # the model we are using returns a multilabel data, when we use
            # this model with 2 labels, it becomes binary.
            if output_labels == 2:
                pred = pred[:,0]
                y = y[:,0]
                y = tf.center_crop(y, pred.shape[1:])
            else:
                y = tf.center_crop(y, pred.shape[2:])
            acc = accuracy(pred, y)
            f1 = f1score(pred, y)
            iou = intersection(pred, y)
            rec = recall(pred, y)
            prec = precision(pred, y)
            spec = specificity(pred, y)
            # calculating loss
            pred = (pred > 0.5).float()
            loss = loss_fn(pred, y)
            # deliting variables
            loss_item = loss.item()
            # freeing up space
            del loss, pred, x, y, image, label, dictionary
    # computing final metric
    acc = accuracy.compute()
    f1 = f1score.compute()
    iou = intersection.compute()
    rec = recall.compute()
    prec = precision.compute()
    spec = specificity.compute()
    # freeing up space
    del loader, loop

    print('\n'+title+f'Got an accuracy of {round(acc.item(), 4)}')
    print('\n'+title+f'F1 score: {round(f1.item(), 4)}'+'\n')
    # resiting metrics
    accuracy.reset()
    f1score.reset()
    intersection.reset()
    recall.reset()
    precision.reset()
    specificity.reset()
    # returning model to training
    model.train()

    return acc.item(), f1.item(), iou.item(), rec.item(), prec.item(), spec.item(), loss_item


#%% Saving images (only if the output are images)

def save_predictions_as_imgs(loader, model, folder='saved_images',
                             device='cuda' if torch.cuda.is_available() else 'cpu',
                             **kwargs):
    # If image is grayscale, if yes, we have to turn into rgb to save
    gray = kwargs.get('gray')
    # With model in evaluation
    model.eval()
    for idx, (dictionary) in enumerate(loader):
        image, label = dictionary
        x, y = dictionary[image], dictionary[label]
        x = x.to(device=device)
        y = y.to(dtype=torch.float32)
        y = y.to(device=device)
        with torch.no_grad():
            pred = model(x)['out']
            y = tf.center_crop(y, pred.shape[2:])
            pred = (pred > 0.5).float()
        # If image is grayscale, transforming to 'rgb' (utils.save_image needs)
        if gray:
            pred = torch.cat([pred,pred,pred],1)
            y = y.unsqueeze(1)
            y = torch.cat([y,y,y],1)
            y = y.float()
            y = tf.center_crop(y, pred.shape[2:])
        save_image(pred, f'{folder}/pred_{idx}.png')
        save_image(y, f'{folder}/y_{idx}.png')

    model.train()


In [ ]:
!pip install google-colab

In [ ]:
# -*- coding: utf-8 -*-
'''
Adversarial Attack on Segmentation Models (UResNets)

This program is intended to be a general tool for training any segmentation
model and test its robustness with any attack strategy. To approach other
models just changes the 'model.py' file, or load the desired model in the va-
riable 'model'. And to change the attack, just change the attack class 'atk'
with the desired attack strategy (the hint is to use Cleverhans or Torchattacks
libraries).

This algorithm uses Torchattacks library to attack a pre-trained model of 97%
accuracy in the segmentation of cytoplasm and nuclei regions in white-blood
cells (WBC) from blood stained slides. Using the Torchattacks library, a
diverse range of attacks can be conducted using this same algorithm, only by
changing the class called at the 'atk' object to the desired attack method.
See Torchattacks documentation for more information.

This program can only run with other three python files (utils.py, model.py and
dataset.py), and with a trained model, which can be trained using the train.py
file. It is interesting to train the model with the same dataset used in the
attacks. Thus, specify the path for this dataset in the list variables in this
algorithm, e.g. 'train_image_dir' (this is the same name used in train.py for
training). All python files are available on GitHub repository (link below).

- 'utils.py': used to define util functions both for attacking and training.
- 'dataset.py': to load images in the DataLoader class of Torch.
- 'model.py': defined the model itself (it can be changed to any desired model)
which in this case can be either ResNets with 18, 34, 50, 101, and 152 layers.
- 'train.py': algorithm that can be used for training any model described in
'model.py'

To attack your mode in the white-blood cells problem, you need to train it on
WBC datasets, like the Raabin-WBC Dataset (used here), using the 'train.py'
file, or on other dataset of your choise.

P.S.: the cell with main function has to be executed separately in the python
console (copy and paste this cell in the console separately)

Find more on the GitHub Repository:
https://github.com/MarlonGarcia/attacking-white-blood-cells

@author: Marlon Rodrigues Garcia
@instit: University of São Paulo
'''

### Program Header

# If running on Colabs, mounting drive
run_on_colabs = True

# if run_on_colabs:
#     # importing Drive
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     # to import add current folder to path (import py files):
#     import sys
#     root_folder = r'/content/gdrive/MyDrive/...'
#     root_model = r'/content/gdrive/MyDrive/...'
#     sys.path.append(root_folder)
# else:
#     root_folder = r'C:\Users\marlo\Downloads\Dresden Dataset\Enviar ao Thales'
#     root_model = r'C:\Users\marlo\Downloads\Dresden Dataset\Enviar ao Thales'

if run_on_colabs:
    # importing Drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # to import add current folder to path (import py files):
    import sys
    root_folder = r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Desenvolvido pelos Pesquisadores/Thales Pimentel Zuanazzi/Segmentação/teste_optimal_cross'
    root_model  = r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Desenvolvido pelos Pesquisadores/Thales Pimentel Zuanazzi/Segmentação/teste_optimal_cross'
    chekpoint_dir = 'my_checkpoint25.pth.tar'
    # root_folder = r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Thales Pimentel Zuanazzi/Segmentação/training_liver'
    # root_model  = r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Thales Pimentel Zuanazzi/Segmentação/training_liver'
    # chekpoint_dir = 'my_checkpoint10.pth.tar'
    # chekpoint_dir = '/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Thales Pimentel Zuanazzi/Segmentação/primeira rodada de treinamentos/training_pancreas/my_checkpoint67.pth.tar'
    sys.path.append(root_folder)
else:
    root_folder = r'C:/Users/marlo/My Drive/College/Biophotonics Lab/Research/Programs/Python/Camera & Image/Surgery RGB/25.09.2023 - Algoritmo Thales'
    root_model = r'C:/Users/marlo/My Drive/College/Biophotonics Lab/Research/Programs/Python/Camera & Image/Surgery RGB/25.09.2023 - Algoritmo Thales'
    # chekpoint_dir = 'my_checkpoint18.pth.tar'

# importing Libraries
import os
os.chdir(root_folder)
# from utils import *
# from model import *
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as tf
import numpy as np
import cv2
# from cleverhans.torch.attacks.projected_gradient_descent import (
#     projected_gradient_descent,
# )

#%% Defining Hyperparameters and Directories

# hyperparameters
epsilons = [0]
numb_steps = 40         # number of steps for attack
step_size = 2/255       # step size for attack
norm = 'inf'            # norm of attack
output_labels = 2       # number of output labels/classes
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 5e-4    # learning rate
batch_size = 2          # batch size
num_workers = 0         # number of workers (smaller or = n° processing units)
clip_train = 1.00       # percentage to clip the train dataset (for tests)
clip_valid = 1.00       # percentage to clip the valid dataset (for tests)
clip_test = 1.00        # percentage to clip the valid dataset (for tests)
valid_percent = 0.15    # use a percent of train dataset as validation dataset
test_percent = 0.15     # a percent from training dataset (but do not excluded)
image_height = 512      # height to crop the image
image_width = 640       # width to crop the image
pin_memory = True       # setting pin memory to 'True'
load_model = True       # 'true' to load a model and test it
save_images = True      # saving example from predicted and original

# defining the paths to datasets
if run_on_colabs:
    train_image_dir = [
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/01',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/02',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/03',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/04',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/05',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/11',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/12',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/13',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/15',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/18',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/20',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/21',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/22',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/24',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/25',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/26',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/28',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/29',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/30',
                       r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/31'
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/03',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/04',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/05',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/06',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/07',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/09',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/11',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/12',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/14',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/16',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/17',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/18',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/19',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/20',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/21',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/22',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/23',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/24',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/25',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/26'
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/01',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/02',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/03',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/04',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/05',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/07',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/08',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/11',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/12',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/16',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/17',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/18',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/20',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/21',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/22',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/24',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/26',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/27',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/29',
                      # r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/pancreas/31'

                       ]
else:
    train_image_dir = [r'C:/Users/marlo/Downloads/Dresden Dataset/abdominal_wall/03']
# defining validation diretory
if run_on_colabs:
    val_image_dir = [r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/22',
                     r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/24',
                     r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/26',]
else:
    val_image_dir = [r'C:/Users/marlo/Downloads/Dresden Dataset/liver/03'#,
                      # r'C:/Users/marlo/Downloads/Dresden Dataset/abdominal_wall/24',
                      # r'C:/Users/marlo/Downloads/Dresden Dataset/abdominal_wall/26'
                      ]
# images in this directory will be used to save examples of the perturbed ones
if run_on_colabs:
    save_image_dir = [r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/liver/03']
else:
    save_image_dir = [r'C:/Users/marlo/Downloads/Dresden Dataset/liver/03']

# # defining the paths to datasets
# if run_on_colabs:
#     train_image_dir = [r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/03']
# else:
#     train_image_dir = [r'C:/Users/marlo/Downloads/Dresden Dataset/abdominal_wall/03']

# # images in this directory will be used to save examples of the perturbed ones
# if run_on_colabs:
#     save_image_dir = [r'/content/gdrive/Shareddrives/Lab. de Óptica Biomédica/Datasets/DSAD/abdominal_wall/24']
# else:
#     save_image_dir = [r'C:/Users/marlo/Downloads/Dresden Dataset/abdominal_wall/24']


#%% Defining Model and Loading DataLoader

# defining the model
model = UResNet34(in_channels=3, num_classes=2).to(device)
# model = UResNet50(in_channels=3, num_classes=2).to(device)
# initializing optimizer
optimizer = optim.Adam(model.parameters())
schedule = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# defining the loss function
loss_fn = nn.CrossEntropyLoss()

# loading the Pre-trained weights for the UResNet50
if load_model:
    # loading checkpoint, if 'cpu', we need to pass 'map_location'
    os.chdir(root_model)
    if device == 'cuda':
        load_checkpoint(torch.load(chekpoint_dir), model)
    else:
        load_checkpoint(torch.load(chekpoint_dir,
                                   map_location=torch.device('cpu')), model)
model.to(device)

#%% Running main() function, where the model will be attacked and evaluated

# initiating variables of metrics to be saved
acc_test = []
f1_test = []
iou_test = []
rec_test = []
prec_test = []
spec_test = []
acc_valid = []
f1_valid = []
iou_valid= []
rec_valid = []
prec_valid = []
spec_valid = []


def norm255(image):
    image = image - np.min(image)
    image = image/(np.max(image)/255)
    return np.array(image, np.uint8)


def main():
    # Here 'output_labels == 2' means binary. We use this conditional because
    # the model we are using returns a multilabel data, when we use this model
    # with 2 labels, it becomes binary.
    # setting the model to evaluation
    model.eval()
    # creating directory to save results
    os.chdir(root_folder)
    # iterating in 'epsilons' to accounts for different epsilon in perturbation
    for epsilon in epsilons:
        ## Saving images - The next 'get_loaders' and the next 'for' loop in
        #'save_loader' are used to save original and perturbed images. This
        # images cannot be saved in the above loops, because the 'get_loaders'
        # lose part of the image informatino due to its normalization
        print('\n\n- Saving Images...\n')
        # Loading the DataLoader
        # loading DataLoaders
        _, _, save_loader = get_loaders(
            train_image_dir=train_image_dir,
            valid_percent=0,
            test_percent=0,
            batch_size=1,
            image_height=image_height,
            image_width=image_width,
            num_workers=num_workers,
            pin_memory=pin_memory,
            val_image_dir=save_image_dir,
            clip_valid=1.0,
            clip_train=1.0,
            clip_test=1.0
        )

        # creating directory to save results
        os.chdir(root_folder)
        try: os.mkdir('saved_images_attack')
        except: pass
        # Defining names of images to be saved (important to save with the
        # original name for further comparison)
        names = [name for name in os.listdir(save_image_dir[0]) if 'image' in name]
        # Sort the image and label names based on the numeric part extracted from filenames
        names.sort(key=lambda x: int(x[5:7]))  # Extract the two-digit number from "imageXX.png"
        print(names)

        masks = [name for name in os.listdir(save_image_dir[0]) if 'mask' in name]
        masks.sort(key=lambda x: int(x[4:6]))  # Extract the two-digit number from "imageXX.png"
        print(masks)
        # using 'tqdm' library to see a progress bar
        loop = tqdm(save_loader)
        for i, dictionary in enumerate(loop):
            image, label = dictionary
            # Extracting the data from dictionary with keys 'image' and 'label'
            x, y = dictionary[image], dictionary[label]
            # Casting to device
            x, y = x.to(device=device), y.to(device=device)
            # Label needs to be float
            y = y.float()
            # changing from one-hot encoding to label (model output is one-hot)
            if output_labels == 2:
                label = y[:,0]
            elif output_labels>2:
                raise NotImplementedError
            else:
                raise ValueError('\n\noutput_labels has to be 2 or greater')
            # Forward-passing the image
            pred = model(x)
            # changing from one-hot encoding to label
            if output_labels == 2:
                pred = pred[:,0]
                # Cropping the label, in case convolutions changed its size
                label = tf.center_crop(label, pred.shape[1:])
            elif output_labels>2:
                label = tf.center_crop(label, pred.shape[2:])
                raise NotImplementedError
            else:
                raise ValueError('\n\noutput_labels has to be 2 or greater')
            # Turning probabilities in 'True' or 'False', then in float values
            pred = (pred > 0.5).float()
            ## Next lines are to actually save the images
            os.chdir(root_folder)
            # The tensor needs to be converted to numpy to be saved
            temp = label.to('cpu',torch.int32).numpy()[0,:,:]
            # Function to de-normalize the tensor to the [0,255] range
            temp = norm255(temp)
            # Saving image with OpenCV
            # cv2.imwrite(f'saved_images_attack/Imagem_{str(i)}_ground_truth.png', temp)
            # Converting tensor to numpy in appropriate form
            temp = pred.to('cpu',torch.int32).numpy()[0,:,:]
            # De-normalizing the values
            temp = norm255(temp)
            # Saving image with OpenCV
            cv2.imwrite(f'saved_images_attack/Imagem_{str(i)}_predicao.png', temp)
            ## The next steps are to load (imread) and save (imwrite) the ori-
            # ginal and ground truth images
            temp = cv2.imread(save_image_dir[0]+'/'+names[i])
            # Next steps are to return the numpy to the tensor format
            temp = temp.astype(float)/np.max(temp)
            temp = torch.Tensor(temp).permute(2,0,1)
            temp = temp[None,:,:,:]
            temp = tf.resize(temp,(image_height,image_width))
            # Returning to the numpy format to be saved as an image
            temp = temp.permute(0, 2, 3, 1).to('cpu').numpy()[0,:,:,:]
            # De-normalization to the image range
            temp = norm255(temp)
            # Saving image
            cv2.imwrite(f'saved_images_attack/Imagem_{str(i)}_x_original.png', temp)

            temp = cv2.imread(save_image_dir[0]+'/'+masks[i])
            # Next steps are to return the numpy to the tensor format
            temp = temp.astype(float)/np.max(temp)
            temp = torch.Tensor(temp).permute(2,0,1)
            temp = temp[None,:,:,:]
            temp = tf.resize(temp,(image_height,image_width))
            # Returning to the numpy format to be saved as an image
            temp = temp.permute(0, 2, 3, 1).to('cpu').numpy()[0,:,:,:]
            # De-normalization to the image range
            temp = norm255(temp)
            # Saving image
            cv2.imwrite(f'saved_images_attack/Imagem_{str(i)}_ground_truth.png', temp)


if __name__ == '__main__':
    main()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

- Loading Checkpoint...


- Saving Images...

image_names: ['image00.png', 'image01.png', 'image02.png', 'image03.png', 'image04.png', 'image05.png', 'image06.png', 'image07.png', 'image08.png', 'image09.png', 'image10.png', 'image11.png', 'image12.png', 'image13.png', 'image14.png', 'image15.png', 'image16.png', 'image17.png', 'image18.png', 'image19.png', 'image20.png', 'image21.png', 'image22.png', 'image23.png', 'image24.png', 'image25.png', 'image26.png', 'image27.png', 'image28.png', 'image29.png', 'image30.png', 'image31.png', 'image32.png', 'image33.png', 'image34.png', 'image35.png', 'image36.png', 'image37.png', 'image38.png', 'image39.png', 'image40.png', 'image41.png', 'image42.png', 'image43.png', 'image44.png', 'image45.png', 'image46.png', 'image47.png', 'image48.png', 'image49.png', 'image50.png', 'image51.png', 'image52.png', 'image53.png'

  0%|          | 0/90 [00:00<?, ?it/s]

<ipython-input-9-e70c224ca631>:260: RuntimeWarning: invalid value encountered in divide
  image = image/(np.max(image)/255)
<ipython-input-9-e70c224ca631>:261: RuntimeWarning: invalid value encountered in cast
  return np.array(image, np.uint8)
<ipython-input-9-e70c224ca631>:260: RuntimeWarning: invalid value encountered in divide
  image = image/(np.max(image)/255)
<ipython-input-9-e70c224ca631>:261: RuntimeWarning: invalid value encountered in cast
  return np.array(image, np.uint8)
<ipython-input-9-e70c224ca631>:260: RuntimeWarning: invalid value encountered in divide
  image = image/(np.max(image)/255)
<ipython-input-9-e70c224ca631>:261: RuntimeWarning: invalid value encountered in cast
  return np.array(image, np.uint8)
<ipython-input-9-e70c224ca631>:260: RuntimeWarning: invalid value encountered in divide
  image = image/(np.max(image)/255)
<ipython-input-9-e70c224ca631>:261: RuntimeWarning: invalid value encountered in cast
  return np.array(image, np.uint8)
<ipython-input-9-e70